In [ ]:
import httpx
from bs4 import BeautifulSoup
import json
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
import re
from pprint import pprint
from datetime import datetime, timedelta
import random

user_agents=  [  "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:125.0) Gecko/20100101 Firefox/125.0",
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.2 Safari/605.1.15",
                "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:123.0) Gecko/20100101 Firefox/123.0",
                "Mozilla/5.0 (iPad; CPU OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1",
                "Mozilla/5.0 (Linux; Android 11; SM-G991B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6422.112 Mobile Safari/537.36",
                "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:123.0) Gecko/20100101 Firefox/123.0" ]
   



# Example usage of rotating User-Agent inside request loop
def get_rotating_headers() :
    return {  "User-Agent":  random.choice(user_agents) ,
            "Accept-Language": "de-DE,de;q=0.9,en;q=0.8",
            "Accept-Encoding": "gzip, deflate, br" ,
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Connection": "keep-alive", }
       
   

In [ ]:
urls = ["https://www.lifepr.de/pressemitteilung/amsel-ev-aktion-multiple-sklerose-erkrankter-landesverband-der-dmsg-in-baden-wuerttemberg-ev/multiple-sklerose-eine-krankheit-die-man-oft-nicht-sieht/boxid/1019745",
"https://www.lifepr.de/pressemitteilung/institut-fuer-qualitaet-und-wirtschaftlichkeit-im-gesundheitswesen-iqwig/versicherteninformation-mit-entscheidungshilfe-zum-lungenkrebs-screening-mittels-niedrig-dosis-computertomografie/boxid/1019831",
"https://www.lifepr.de/pressemitteilung/staedtisches-klinikum-karlsruhe-gmbh/klinikum-karlsruhe-stellt-weichen-fr-die-zukunft-professor-martin-holderried-wird-neuer-medizinischer-geschftsfhrer/boxid/1019825",
"https://www.lifepr.de/pressemitteilung/verband-der-privatkrankenanstalten-in-bayern-ev/gemeinsames-handeln-im-sinne-der-patientenversorgung/boxid/1019559",
"https://www.lifepr.de/pressemitteilung/landesaerztekammer-hessen/wir-brauchen-eine-strkung-der-ambulanten-patientenversorgung/boxid/1019558",       
"https://www.lifepr.de/pressemitteilung/reha-zentren-passauer-wolf-gmbh/parkinson-kliniken-neu-vernetzt-passauer-wolf-untersttzt-interdisziplinren-austausch-mit-parklink/boxid/1019456",
"https://www.lifepr.de/pressemitteilung/kreisausschuss-des-vogelsbergkreises/sprechtag-des-sprachheilbeauftragten-joachim-eisentrger/boxid/1019315",
"https://www.lifepr.de/pressemitteilung/mi-klinikgruppe-enzensberg/neue-chefrztin-in-der-geriatrie/boxid/1019278",
"https://www.lifepr.de/pressemitteilung/big-direkt-gesund-dortmund/big-direkt-gesund-verdreifacht-zuschuss-zur-professionellen-zahnreinigung-auf-150-eurojhrlich/boxid/1019265",
"https://www.lifepr.de/pressemitteilung/aktion-gesunder-ruecken-agr-ev/rckengesund-von-anfang-an/boxid/1018823",


"https://www.lifepr.de/pressemitteilung/marketing-und-tourismus-konstanz-gmbh/schritt-fr-schritt-im-wandergenuss-premiumwandern-am-westlichen-bodensee/boxid/1019426",
"https://www.lifepr.de/pressemitteilung/kur-tourismus-und-wirtschaftsbetriebe-der-stadt-bad-harzburg-gmbh/wanderung-zum-elfenstein/boxid/1019667",      
"https://www.lifepr.de/pressemitteilung/theater-heilbronn-0/lebenslgen-und-gewissensbisse/boxid/1019737",   
"https://www.lifepr.de/pressemitteilung/greensign-institut-gmbh/hospitality-neu-denken-beim-greensign-future-lab-2025-in-berlin/boxid/1019573",
"https://www.lifepr.de/pressemitteilung/tourismus-agentur-nordsee-gmbh/filzkurse-fr-erwachsene-und-kinder-im-moorinformationszentrum/boxid/1019689",
"https://www.lifepr.de/pressemitteilung/kc3-kpers-corporate-crisis-communications-gmbh/america-unlimited-erweitert-route-66-angebot/boxid/1019695",
"https://www.lifepr.de/pressemitteilung/ktg-karlsruhe-tourismus-gmbh/lesung-aus-das-geheimnis-der-hohen-eich-am-23-april-2025-in-der-tourist-information-im-schaufenster-karlsruhe/boxid/1019464",
"https://www.lifepr.de/pressemitteilung/muthesius-kunsthochschule/maria-gerbaulet-erhlt-den-bundespreis-fr-kunststudierende/boxid/1019516",
"https://www.lifepr.de/pressemitteilung/erzgebirgische-theater-orchester-gmbh-annaberg-buchholz/erffnung-der-fotoausstellung-doorstep-paradise/boxid/1019566",
"https://www.lifepr.de/pressemitteilung/sprengel-museum-hannover/spectrum-internationaler-preis-fr-fotografie-der-stiftung-niedersachsen-frida-orupabo-im-sprengel-museum-hannover-von-5-4-bis-20-7-25/boxid/1019642",


"https://www.lifepr.de/pressemitteilung/whirlpool-center-gmbh/profisportler-regenerieren-im-outdoor-whirlpool/boxid/1019810",      
"https://www.lifepr.de/pressemitteilung/bad-bevensen-marketing-gmbh/tagesangebote-mai-2025/boxid/1019225", 
"https://www.lifepr.de/pressemitteilung/wunderland-kalkar-0/wunderland-kalkar-freizeitpark-startet-am-5-april-in-die-neue-saison/boxid/1017898",
"https://www.lifepr.de/pressemitteilung/mankau-verlag-gmbh/fitness-fr-frauen/boxid/1019674",
"https://www.lifepr.de/pressemitteilung/motorsport-arena-oschersleben-gmbh/finale-vorbereitungsphase-der-dtm-in-der-motorsport-arena-oschersleben/boxid/1019624",
"https://www.lifepr.de/pressemitteilung/wir-sind-wir-inclusion-in-sailing-ev/segel-setzen-fr-mehr-gelebte-inklusion/boxid/1019436",
"https://www.lifepr.de/pressemitteilung/xenofit-gmbh/unterwegs-in-den-bergen-mit-den-kohlenhydrat-produkten-von-xenofit/boxid/1019696",
"https://www.lifepr.de/pressemitteilung/velokonzept-saade-gmbh/was-berlin-bewegt-festival-veloberlin-feiert-das-fahrrad/boxid/1019605",
"https://www.lifepr.de/pressemitteilung/freizeitzentrum-xanten-gmbh/saisonstart-30-mrz-wassersport-und-freizeitvergngen-an-der-xantener-nord-und-sdsee/boxid/1018840",
"https://www.lifepr.de/pressemitteilung/troika-germany-gmbh/keine-kompromisse-dein-perfekter-outdoor-begleiter/boxid/1017676",


"https://www.lifepr.de/pressemitteilung/pearlgmbh/auvisio-2in1-audio-transmitter-und-receiver-mit-bluetooth-5-3-und-display/boxid/1019478",
"https://www.lifepr.de/pressemitteilung/pearlgmbh/rosenstein-shne-elektro-ganzfrucht-kalt-entsafter-800ml-rcklaufgang-60-u-min-200-watt/boxid/1019177",
"https://www.lifepr.de/pressemitteilung/pearlgmbh/auvisio-faltbarer-reise-over-ear-kopfhrer-ohs-400-bt-mit-akku-anc-bt-5-3-mikro/boxid/1018728",
"https://www.lifepr.de/pressemitteilung/disq-deutsches-institut-fuer-service-qualitaet-gmbh-co-kg/deutscher-app-award-2025/boxid/1014798",
"https://www.lifepr.de/pressemitteilung/pearlgmbh/sichler-haushaltsgerte-dampf-waschsauger-fr-polster-textilien-autositze-1350-watt-12-kpa/boxid/1010799",
"https://www.lifepr.de/pressemitteilung/pearlgmbh/auvisio-2-1-soundbar-msx-460-bt-mit-integriertem-subwoofer-180-watt-bluetooth-5-hdmi-aux/boxid/1010678",
"https://www.lifepr.de/pressemitteilung/sauter-deutschland/produkteinfhrung-neue-motorantriebe-fr-stellgerte/boxid/1009635",
"https://www.lifepr.de/pressemitteilung/pearlgmbh/vr-radio-digitales-dab-radio-dor-220-2-4-6-1-cm-farbdisplay-bluetooth-holzgehuse/boxid/1008758",
"https://www.lifepr.de/pressemitteilung/pearlgmbh/generalkeys-optische-silent-funk-maus-mit-akku-fm-1650-nano-receiver-bis-1-600-dpi/boxid/1008457",
"https://www.lifepr.de/pressemitteilung/pearlgmbh/rosenstein-shne-3in1-akku-handmixer-zerkleinerer-und-schneebesen-usb-20-watt/boxid/1008291",


"https://www.lifepr.de/pressemitteilung/winter-fahrzeugtechnik-gmbh/elektrisch-effizient-im-verteilerverkehr-winter-khlfahrzeuge-rstet-als-erster-den-ford-e-transit-custom-zum-khlfahrzeug-um/boxid/1019709",
"https://www.lifepr.de/pressemitteilung/freiburger-verkehrs-ag/jahresbilanz-der-lngsten-umlaufseilbahn-deutschlands-erfreuliche-fahrgastzahlen-digitale-tickets-und-krzere-wartungsintervalle-der-kabinen/boxid/1019834",       
"https://www.lifepr.de/pressemitteilung/jms-fahrzeugteile-gmbh/cor-speed-meets-tridente-gmbh-jaguar-f-type-auf-deville-rdern/boxid/1019838", 
"https://www.lifepr.de/pressemitteilung/peugeot-deutschland-gmbh/peugeot-zur-beliebtesten-marke-der-franzosen-in-der-kategorie-autohersteller-gewhlt/boxid/1019734",
"https://www.lifepr.de/pressemitteilung/cambio-mobilitaetsservice-gmbh-cokg-bremen/erfolgreiches-carsharing-jahr-2024-umsatz-von-anbieter-cambio-wchst/boxid/1019660",
"https://www.lifepr.de/pressemitteilung/kraftfahrer-schutz-ev-muenchen/ja-zu-helmen-auf-e-scootern-ja-zu-einer-besseren-infrastruktur/boxid/1019602",
"https://www.lifepr.de/pressemitteilung/opel-automobile-gmbh/mehr-als-was-frs-auge-der-neue-opel-vivaro-electric-sportive/boxid/1019568",
"https://www.lifepr.de/pressemitteilung/q-park-gmbh-co-kg/q-park-kooperiert-mit-connected-mobility-dsseldorf-fr-smartes-feierabend-parken/boxid/1019257",
"https://www.lifepr.de/pressemitteilung/cambio-mobilitaetsservice-gmbh-cokg-bremen/25-jahre-erfolgreiches-carsharing-unter-der-marke-cambio/boxid/1018645",
"https://www.lifepr.de/pressemitteilung/freiburger-verkehrs-ag/fundsachenversteigerung/boxid/1018580",


"https://www.lifepr.de/pressemitteilung/kreisausschuss-des-vogelsbergkreises/schuljahr-der-nachhaltigkeit-an-der-eichbergschule/boxid/1019815",     
"https://www.lifepr.de/pressemitteilung/paedagogische-hochschule-karlsruhe/neues-phka-weiterbildungsangebot-lernen-und-arbeiten-im-austausch-der-generationen/boxid/1019936",     
"https://www.lifepr.de/pressemitteilung/hochschule-osnabrck/2-55-millionen-euro-fr-die-frderung-wissenschaftlicher-startups-der-osnabrcker-hochschulen/boxid/1019539",      
"https://www.lifepr.de/pressemitteilung/hochschule-weserbergland/neuer-dualer-studiengang-soziale-arbeit-startet/boxid/1018104",
"https://www.lifepr.de/pressemitteilung/paedagogische-hochschule-karlsruhe/politische-medienbildung-prof-dr-sren-torrau-forscht-und-lehrt-jetzt-an-der-pdagogischen-hochschule-karlsruhe/boxid/1017741",
"https://www.lifepr.de/pressemitteilung/paedagogische-hochschule-karlsruhe/praxis-im-lehramtsstudium-lehr-lern-labor-physik-der-pdagogischen-hochschule-karlsruhe-erhlt-anschlussfrderung/boxid/1019264",
"https://www.lifepr.de/pressemitteilung/fql-agentur-fuer-kommunikationsmanagement/wissen-wollen-knnen-tun-der-vortrag-der-kpfe-rttelt-denkmuster-schttelt-und-ins-handeln-bringt/boxid/1019978",
"https://www.lifepr.de/pressemitteilung/hochschule-worms/erfolgreiche-prsentationen-auf-der-ivapp-konferenz-in-porto-2025/boxid/1019874",
"https://www.lifepr.de/pressemitteilung/hochschule-worms/wie-generative-ki-und-analytics-wirtschaft-die-arzneimittelversorgung-revolutionieren-knnen/boxid/1017597",
"https://www.lifepr.de/pressemitteilung/paedagogische-hochschule-karlsruhe/biographische-forschung-der-lteren-generation-eine-stimme-geben/boxid/1015154",



"https://www.lifepr.de/pressemitteilung/devk-deutsche-eisenbahn-versicherung/devk-umfrage-zeigt-investitionen-in-hochwasserschutz-sind-gefragt/boxid/1019453",      
"https://www.lifepr.de/pressemitteilung/makler-nachfolger-club-e-v/erster-deutscher-makler-nachfolger-club-e-v-begrt-sein-1-000-mitglied/boxid/1019266",      
"https://www.lifepr.de/pressemitteilung/universa-versicherungen/wachstum-in-allen-unternehmen-universa-entwickelt-sich-kontinuierlich-weiter/boxid/1019467",     
"https://www.lifepr.de/pressemitteilung/disq-deutsches-institut-fuer-service-qualitaet-gmbh-co-kg/deutscher-versicherungs-award-2025/boxid/1019494", 
"https://www.lifepr.de/pressemitteilung/bca-ag/comparit-erweitert-gesellschafterkreis-mit-der-bca-ag/boxid/1019263",
"https://www.lifepr.de/pressemitteilung/sparkasse-bremen-ag/sparkasse-bremen-sucht-nachwuchskrfte-per-speed-recruiting-in-30-minuten-einen-ausbildungsvertrag-in-der-tasche-haben/boxid/1019031",
"https://www.lifepr.de/pressemitteilung/cllb-rechtsanwaelte/anlagerepublik-betrug-mit-festgeldanlagen/boxid/1018569",
"https://www.lifepr.de/pressemitteilung/cllb-rechtsanwaelte/yavita-invest-ag-betrug-mit-festgeldanlagen/boxid/1018567",
"https://www.lifepr.de/pressemitteilung/cllb-rechtsanwaelte/strunz-kapital-com-betrug-mit-festgeldanlagen/boxid/1018564",
"https://www.lifepr.de/pressemitteilung/mph-legal-services/die-ehegattenbrgschaft-im-spannungsfeld-von-haftung-scheidung-und-finanzieller-leistungsfhigkeit/boxid/1018548",

"https://www.lifepr.de/pressemitteilung/artefact-ggmbh/erfolgreicher-saisonstart-im-klimapark-geladene-gste-weihten-deutschlands-grte-klimawand-ein/boxid/1019861",      
"https://www.lifepr.de/pressemitteilung/flyeralarm-gmbh/klima-coaching-frderung-fr-den-mittelstand/boxid/1018517",      
"https://www.lifepr.de/pressemitteilung/deutsches-energieberater-netzwerk-den-ev-frankfurt-am-main/ein-berufsbild-fr-die-zukunft-fr-qualifizierte-energieberatung/boxid/1018448",      
"https://www.lifepr.de/pressemitteilung/bundesamt-fuer-naturschutz/gemeinsam-mee-h-r-natur-wagen/boxid/1017772",
"https://www.lifepr.de/pressemitteilung/abenteuer-regenwald-ev/aktionstag-palml-was-macht-der-orang-utan-in-meinem-einkaufswagen/boxid/1017840",
"https://www.lifepr.de/pressemitteilung/kommunale-arbeitsgemeinschaft-dialogforum-airport-berlin-brandenburg/biodiversittskonzept-fr-treptow-kpenick/boxid/1017342", 
"https://www.lifepr.de/pressemitteilung/vitra-bad-gmbh/ein-groer-schritt-von-vitra-fliesen-fr-eine-nachhaltige-zukunft/boxid/1017330",
"https://www.lifepr.de/pressemitteilung/deutsche-stiftung-meeresschutz-dsm-neuss/zum-tag-des-artenschutzes-am-3-mrz-un-world-wildlife-day/boxid/1016094",
"https://www.lifepr.de/pressemitteilung/deutsche-stiftung-meeresschutz-dsm-neuss/zum-welttag-des-seegrases-2025-am-1-mrz-ein-meereskosystem-der-superlative-in-gefahr/boxid/1015982",
"https://www.lifepr.de/pressemitteilung/deutsche-stiftung-meeresschutz-dsm-neuss/am-22-mrz-ist-der-internationale-tag-der-robben/boxid/1018314",


"https://www.lifepr.de/pressemitteilung/lbs-landesbausparkasse-nordost-ag/aktueller-lbs-immobilienmarktatlas-hamburg-und-umland-2025/boxid/1019419",    
"https://www.lifepr.de/pressemitteilung/schlueter-systems-kg/mehr-wohnraum-im-dach-neue-taskforce-entwickelt-skalierbare-lsungen-fr-den-dachausbau/boxid/1019212",      
"https://www.lifepr.de/pressemitteilung/heinze-gmbh/neue-modernisierungsstudie-macht-den-wohn-nichtwohnbau-transparent-und-messbar/boxid/1019818", 
"https://www.lifepr.de/pressemitteilung/gpep-gmbh/gpep-erwirbt-edeka-portfolio-in-bayern/boxid/1017097",
"https://www.lifepr.de/pressemitteilung/verlagsgruppe-rudolf-mueller/immobilienmanager-award-2025-die-besten-ideen-der-immobilienbranche-prmiert/boxid/1016801",
"https://www.lifepr.de/pressemitteilung/union-investment-real-estate-ag-business-network/zukunft-des-bros-neue-studie-von-momeni-gruppe-und-union-investment-zeigt-was-broflchen-attraktiv-macht/boxid/1016021",
"https://www.lifepr.de/pressemitteilung/gpep-gmbh/geschftsjahr-2024-gpep-bleibt-auf-wachstumskurs/boxid/1015057",
"https://www.lifepr.de/pressemitteilung/landesinnungsverband-des-dachdeckerhandwerks-rheinland-pfalz/ist-der-lack-ab/boxid/1019133",
"https://www.lifepr.de/pressemitteilung/fbf-betondienst-gmbh/69-betontage-geben-wertvolle-impulse-fr-die-betonbranche/boxid/1019359",
"https://www.lifepr.de/pressemitteilung/eurolam-gmbh/eurolam-referenzseite-erstrahlt-in-neuem-glanz-entdecken-sie-innovative-projekte/boxid/1018308",


"https://www.lifepr.de/pressemitteilung/kreisausschuss-des-vogelsbergkreises/fachstelle-gegen-sexualisierte-gewalt-jetzt-bei-vogelsberger-lebensrume/boxid/1019777",      
"https://www.lifepr.de/pressemitteilung/caritasverband-der-dizese-rottenburg-stuttgart-e-v/wohnungslosigkeit-dauerhaft-berwinden/boxid/1019390",
"https://www.lifepr.de/pressemitteilung/stiftung-haensel-gretel/karoline-herfurth-ehrt-dr-christine-bergmann-mit-deutschem-kinderschutzpreis/boxid/1018947",       
"https://www.lifepr.de/pressemitteilung/vereinte-evangelische-mission/dokumentation-der-dritten-interreligisen-friedenskonferenz-erscheint-am-26-mrz/boxid/1018293",
"https://www.lifepr.de/pressemitteilung/landeszentrale-fuer-politische-bildung-baden-wuerttemberg/religion-und-ffentlichkeit-die-bedeutung-religiser-akteure-in-politik-und-gesellschaft/boxid/1018267",
"https://www.lifepr.de/pressemitteilung/samaritans-purse-e-v/franklin-graham-warnt-vor-verrohung-der-sprache-besorgniserregender-trend-auch-in-deutschland/boxid/1018147",
"https://www.lifepr.de/pressemitteilung/landeszentrale-fuer-politische-bildung-baden-wuerttemberg/demokratie-in-der-krise-wirtschaft-handelt/boxid/1017960",
"https://www.lifepr.de/pressemitteilung/giordano-bruno-stiftung/selbst-cdu-whler-halten-die-kirchen-fr-unwichtig/boxid/1016546",
"https://www.lifepr.de/pressemitteilung/freikirche-der-siebenten-tags-adventisten-in-deutschland-nachrichtenagentur-apd/das-recht-auf-gedanken-gewissens-und-religionsfreiheit-ist-nicht-verhandelbar/boxid/1015834",
"https://www.lifepr.de/pressemitteilung/stiftung-unionhilfswerk-berlin/danke-fr-die-unvergesslichen-momente-jubilumskampagne-der-unionhilfswerk-frderstiftung-erfolgreich-beendet/boxid/1015236",


"https://www.lifepr.de/pressemitteilung/barnhouse-naturprodukte-gmbh/barnhouse-erfllt-kundenwnsche-zwei-ganz-neue-bio-knuspermslis/boxid/1013173",
"https://www.lifepr.de/pressemitteilung/barnhouse-naturprodukte-gmbh/bee-wild-2025-barnhouse-sucht-50-engagierte-wildbienen-hoteliers/boxid/1015379",
"https://www.lifepr.de/pressemitteilung/backshop-tiefkuehl-gmbh/backshop-tiefkhl-gmbh-erweitert-vertrieb-mit-martins-potato-rolls-in-europa/boxid/1019420",
"https://www.lifepr.de/pressemitteilung/brauerei-gasthof-kundmueller-gmbh/deutschlands-bester-nachwuchsbrauer-wurde-bei-der-brauerei-kundmller-in-weiher-ausgebildet/boxid/1006099",
"https://www.lifepr.de/pressemitteilung/byodo-naturkost-gmbh/auszeichnung-fr-spitzenqualitt/boxid/1007996",
"https://www.lifepr.de/pressemitteilung/fauser-vitaquell-gmbh-cokg/fnf-auf-einen-streich-cremig-pikant-lecker/boxid/1015940",
"https://www.lifepr.de/pressemitteilung/fauser-vitaquell-gmbh-cokg/starte-gesund-ins-jahr-veganuary-leicht-gemacht/boxid/1009989",
"https://www.lifepr.de/pressemitteilung/iii-freunde-gmbh/joko-und-matthias-prsentieren-mit-iii-freunde-eine-besondere-limited-edition/boxid/1019886",
"https://www.lifepr.de/pressemitteilung/winzergenossenschaft-oberbergen-eg/neue-weinlinie-mit-grauburgunder-fein-fruchtig/boxid/1013697",
"https://www.lifepr.de/pressemitteilung/winzergenossenschaft-oberbergen-eg/island-of-volcanic-heritage-burgundy-wines-crafted-at-the-perfect-place-since-1271/boxid/1017454",


    
]


branche_pick=["Gesundheit & Medizin",
"Gesundheit & Medizin","Gesundheit & Medizin",
              "Gesundheit & Medizin","Gesundheit & Medizin","Gesundheit & Medizin",
              "Gesundheit & Medizin","Gesundheit & Medizin","Gesundheit & Medizin",
              "Gesundheit & Medizin",
              
              
        "Kultur & Tourismus & Urlaub", "Kultur & Tourismus & Urlaub", "Kultur & Tourismus & Urlaub",
                 "Kultur & Tourismus & Urlaub", "Kultur & Tourismus & Urlaub", "Kultur & Tourismus & Urlaub",
                 "Kultur & Tourismus & Urlaub", "Kultur & Tourismus & Urlaub", "Kultur & Tourismus & Urlaub",
                 "Kultur & Tourismus & Urlaub", 
    
            
  
        "Sport & Freizeit & Unterhaltung", "Sport & Freizeit & Unterhaltung", "Sport & Freizeit & Unterhaltung",
              "Sport & Freizeit & Unterhaltung", "Sport & Freizeit & Unterhaltung", "Sport & Freizeit & Unterhaltung",
              "Sport & Freizeit & Unterhaltung", "Sport & Freizeit & Unterhaltung", "Sport & Freizeit & Unterhaltung",
              "Sport & Freizeit & Unterhaltung",

              
              
         "Technik & Elektronik", "Technik & Elektronik", "Technik & Elektronik",
              "Technik & Elektronik", "Technik & Elektronik", "Technik & Elektronik", 
              "Technik & Elektronik", "Technik & Elektronik", "Technik & Elektronik", 
              "Technik & Elektronik", 

              
         "Verkehr & Logistik",
"Verkehr & Logistik","Verkehr & Logistik",
          "Verkehr & Logistik","Verkehr & Logistik","Verkehr & Logistik",
          "Verkehr & Logistik","Verkehr & Logistik","Verkehr & Logistik",
          "Verkehr & Logistik",
              
         "Bildung & Wissenschaft",
"Bildung & Wissenschaft","Bildung & Wissenschaft",
              "Bildung & Wissenschaft","Bildung & Wissenschaft","Bildung & Wissenschaft",
              "Bildung & Wissenschaft","Bildung & Wissenschaft","Bildung & Wissenschaft",
              "Bildung & Wissenschaft",
              
         "Finanzen & Versicherung",
"Finanzen & Versicherung","Finanzen & Versicherung",
              "Finanzen & Versicherung",
"Finanzen & Versicherung","Finanzen & Versicherung",
              "Finanzen & Versicherung","Finanzen & Versicherung","Finanzen & Versicherung",
              "Finanzen & Versicherung",
              
              
         "Energie & Umwelt",
"Energie & Umwelt","Energie & Umwelt",
              "Energie & Umwelt","Energie & Umwelt","Energie & Umwelt",
              "Energie & Umwelt","Energie & Umwelt","Energie & Umwelt",
              "Energie & Umwelt",
              
              
         "Immobilien & Bauen", "Immobilien & Bauen", "Immobilien & Bauen",
              "Immobilien & Bauen", "Immobilien & Bauen", "Immobilien & Bauen",
              "Immobilien & Bauen", "Immobilien & Bauen", "Immobilien & Bauen",
              "Immobilien & Bauen", 

              
         "Politik & Gesellschaft",
"Politik & Gesellschaft","Politik & Gesellschaft",
               "Politik & Gesellschaft","Politik & Gesellschaft","Politik & Gesellschaft",
               "Politik & Gesellschaft","Politik & Gesellschaft","Politik & Gesellschaft",
               "Politik & Gesellschaft",

              
              "Essen & Trinken", 
              "Essen & Trinken",   "Essen & Trinken",   "Essen & Trinken", 
                "Essen & Trinken",   "Essen & Trinken",   "Essen & Trinken", 
                "Essen & Trinken",   "Essen & Trinken",   "Essen & Trinken", ]
         

        


training_data= []
for link , branche in zip( urls, branche_pick ):
    
    response= httpx.get(link, headers= get_rotating_headers(), timeout= httpx.Timeout(30.0) )

    soup = BeautifulSoup( response.text , 'html.parser')
    
    article_body =soup.find("div", {"itemprop": "articleBody"} )
    text = article_body.get_text(strip =True)   if article_body else ""
    
    keyword_block= soup.find( "ol", {"itemprop": "keywords" } )
    schlagwoerter= [ span.text.strip()  for span in keyword_block.find_all("span") ]   if keyword_block else []
    
    kategorien_block=soup.find( "ol" , class_ ="list-unstyled mb-0 row g-1")
    kategorien=[]
    if  kategorien_block:
        for li  in kategorien_block.find_all( "li") :
            span= li.find( "span" , itemprop= "articleSection")
            if span:
                value= span.get_text( strip =True )
                if  value.lower() !=  "pressemitteilung" :
                    kategorien.append( value )

    training_data.append({    "text": text,
                            "Schlagwörter": ", ".join(schlagwoerter),
                            "branche": branche,
                            "Kategorien": ", ".join(kategorien),
                            "link": link  })
     
    
    time.sleep(3)
    
with open( "training_data.jsonl", "w" ,encoding="utf-8") as f:
    for item  in training_data:
        f.write(json.dumps(item , ensure_ascii=False) + "\n")